## AI Travel Agent: Organising trips and finding best places to visit 🚀
This notebook is a Concierge multi-agent Agent AI assistant to suggest travel options for both leisure and business purposes using Google's ADK.

🎯 Objective: Ease the process and save time before finalising bookings.

Capstone Project is part of the 5-Day AI Agents Intensive Course with Google (Nov 10 - 14, 2025).


![image.png](attachment:ae6da760-3b1e-4990-8207-b3abfe00e5eb.png)

🌟 Key Features

- ✅ ***Multi-agent*** system architecture.

       * Search for flights based on destination
       * Searching for flight prices
       * Rank the top 3 cheapest flights
       * Parallel agent system framework to search for hotels annd prices - happens simultaneously with flight search.
       * Loop refinement agent to reinfe the output from the agent and not relyin on one-shot response: Flight Assistant, Price and hotel scouter → Orchestraror-LLM
       * Sequential Agent to manage and coordinate the agents workflow
       * Powered Coordinator: Root agent orchestrates the entire process → Final Suggestion
- ✅Custom-Function Tools
- ✅Save country, hotel, flihgts
- Retrieve information
- ✅Check from the traveller his/her budget, pause based on the output and then finalises the suggestion.
- ✅Memory and Database storage for using session and states. Hold the memory of the proposed suggestion and trip of the user, needed if changing plans, add database using sqlite (long-term memory with databases).


Workflow:

![miro-flow-chart.png](attachment:9585e941-26a4-4105-9bb4-7c1bf7883c13.png)



## Section 1: Setup ⚙️
Install any dependencies if not running within Kaggle Notebooks i.e. [google-adk](http://) library for Python, `!pip install google-adk`, configure Gemini API Key and performing authentication.

In [ ]:
#!pip install google-adk, ADK

### 1.1 Gemini API Key configuration

In [ ]:
# Configure the Gemini API key
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: 'GOOGLE_API_KEY' failed to be added to the Kaggle secrets. Details: {e}")



### 1.2 Import Agent Development Kit components and Generative AI libs.

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent,LlmAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService, DatabaseSessionService
from google.adk.memory import InMemoryMemoryService

from google.adk.tools import load_memory, preload_memory
from google.adk.tools.tool_context import ToolContext
from google.genai.types import Content, Part


from google.genai import types
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.models.google_llm import Gemini

from typing import Any, Dict
from google.adk.runners import Runner

print("✅ ADK components imported successfully.")

### 1.3 Configure Retry Options

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Section 2: An AI multi-agent architecture for flight and hotel scanning  🤖🤖🤖
- Initially started with a very simple agent and then build a multi-agent framework so that they can find available flights, cheapest and hotels to stay

### 2.1 A team of specialists - Agent Architecture
- `flight_assistant`: search google for flight companies
- `price_agent`:  find the 3 cheapest flights from UK
- `hotel_scouter`: an agent to search hotels
- Upgrading to a parallel workflow with a multi agent architecture.
- A coordinate-sequential agent manages x3 sub-agents

Working with parrallel agents, that are nested inside a sequential agent and combine:
- Flight assistant sub-agent runs first
- Price sub-agentagent provides some options
- Hotel scouter sub agent find options to book for hotels
- The agents are combined under a parallel agent, which is itself nested inside of a sequential agent.

### 2.2 Adding a loop agent to refine the search

- The previous was one-shot quality and needs refinement the final suggestions should be good enough.
  
--> Iterative approach is used using a loopAgent

Bring the agents together under a loop agent, which  which is itself nested inside of a sequential agent.

## Section 3. Working with session state and tools
- Use `tool_context.state` to read/write session state
- Use descriptive key prefixes (`user`, `country`, `app:`) for organization

In [ ]:
async def run_session_2(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

In [ ]:
# Create tool functions to the Agent for the database sql
def save_userinfo(
    tool_context: ToolContext, origin: str, destination: str,
    date_from:str, date_to: str
) -> Dict[str, Any]:
    """
    Tool to record and save user origin and destination city or country and the dates from and to in session state.

    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["origin"] = origin
    tool_context.state["destination"] = destination
    tool_context.state["date_from"] = date_from
    tool_context.state["date_to"] = date_to

    return {"status": "success"}

def retrieve_flight_info(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user preferences for city origin, destination and dates from and to from session state.
    """
    # Read from session state
    origin = tool_context.state.get("user:origin", "Origin not found")
    destination = tool_context.state.get("user:destination", "Destination not found")
    date_from = tool_context.state.get("user:date_from", "Date from not found")
    date_to = tool_context.state.get("user:date_to", "Date to not found")

    return {"status": "success", "origin": origin, "destination": destination,
            "date_from": date_from, "date_to": date_to
           }

print("✅ Tools created.")

In [ ]:
#--- ARCHITECTURE OF AGENTS ---
# Create a persistent memory service. In a real app, and back this
# with a vector DB, but here initialise the standard service.


# --Constants--
USER_ID ="default"
#SESSION = "default"
APP_NAME = "MemoryDemoApp"
MODEL_NAME="gemini-2.5-flash-lite"
model_config = Gemini(model=MODEL_NAME, retry_options=retry_config)

# STEP 1: Initial flight_assistant, price_agent, hotel_scouter Agents (Runs ONCE)
#Agent Architecture:
# Flight Assistant
flight_assistant = Agent(
    model=MODEL_NAME,
    name="FlightAssistant",
    instruction="""
    You are a flight search specialist.
    1. Search google for flights matching the user's request.
    2. Return the top 3 cheapest options and include airline, airport of origin, airport of destination and indicative flight price.
    3. Restrict findings to 3 bullet points
    """,
    tools=[google_search],
    output_key="flight_search"
)

flight_assistant_tool = AgentTool(agent=flight_assistant)

# Hotels Finder
hotel_finder = Agent(
    name="HotelFinder",
    model="gemini-2.5-flash-lite",
    instruction="""
    You are a hotel finder assistant.
    1. Search google for hotels matching the user's destination information.
    2. Return the cheapest and most preferable hotels in terms of 2 stars, 3 stars, and 4 stars. Include price, hotel name and address.""",
    tools=[google_search],
    output_key="hotel_search"
)

hotel_finder_tool = AgentTool(agent=hotel_finder)

# Concierge-Manager-Aggregator Agent:
manager_agent = LlmAgent(
    model=model_config,
    name="ManagerAgent",
    instruction="""
    You are a trip itinerary coordinator assistant.

    When users request a flight itinerary:
    1. Parse the user's request to extract origin, destination, date_from, and date_to, use save_userinfo tool to save the info,
    then use retrieve_flight_info tool to retrieve the information.
    2. USE the 'flight_assistant_tool' to search the information about the flights and provide three bullet points.
    3. USE the 'hotel_finder_tool' to search the information regarding the hotels and provide the 3 bullet points.
    """,
    tools=[save_userinfo, retrieve_flight_info, flight_assistant_tool,
           hotel_finder_tool]
)


print("✅ refiner_agent aggregator created.")

# The ParallelAgent runs all its sub-agents simultaneously.
parallel_team = ParallelAgent(
    name="ParallelTeam",
    sub_agents=[flight_assistant, hotel_finder],
)
# STEP 2: Refiner/Exiter Agent
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
travel_refinement_loop = LoopAgent(
    name="TravelRefinementLoop",
    sub_agents=[parallel_team,manager_agent],
    #sub_agents=[refiner_agent],
    max_iterations=2,  # Prevents infinite loops
    #tools=[
    #    FunctionTool(exit_loop)
    #],  # The tool is now correctly initialized with the function reference.
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="SequentialAgent",
    sub_agents=[travel_refinement_loop],
)



# Memory recall agent
memory_recall_agent = Agent(
    model=model_config,
    name="MemoryRecallAgent",
    instruction="Use the 'load_memory' tool "
                "the answer might be in past conversations.",
    tools=[load_memory]
)

print("✅ Flight Agent created!")
print("✅ Hotel finder Agent created!")
print("✅ Manager Agent created!")
print("✅ Parallel Agent created!")
print("✅ Travel refine Agent created!")
print("✅ Sequential Agent created!")
print("✅ Memory Recall Agent created!")


In [ ]:
# Step 2: Set up Session Management and Add Memory to the Agent
# InMemorySessionService stores conversations in RAM (temporary)
#session_service = InMemorySessionService()
#runner = InMemoryRunner(agent=concierge_agent)

# Switch to DatabaseSessionService
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)
# --- INITIALIZE MEMORY SERVICE
#memory_service = (
#     InMemoryMemoryService()
# )
memory_service= InMemoryMemoryService()

In [ ]:
async def run_scenario():
    runner1 = Runner(
        agent=root_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service) # Memory service is now available!

    print("✅ Agent and Runner created with memory support!")
    print(f"   - Application: {APP_NAME}")
    print(f"   - Using: {session_service.__class__.__name__}")
    #print(f"   - Session: {SESSION}")

    sessionID = "s12"
    await runner1.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=sessionID)
    user_input1 = Content(parts=[Part(text="Hi, my travel plans are: origin:Manchester,  destination:Italy, date_from: 1st of April 2026 date_to: 10th of April 2026")], role="user")

    # Run the agent
    final_response_text = "(no final response)"
    async for event in runner1.run_async(user_id=USER_ID, session_id=sessionID, new_message=user_input1):
            if event.is_final_response() and event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
    print(f"Agent 1 Response: {final_response_text}")

    # Get the completed session
    completed_session1 = await runner1.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=sessionID)

    # Add this session's content to the Memory Service
    print("\n--- Adding Session 1 to Memory ---")
    await memory_service.add_session_to_memory(completed_session1)
    print("Session added to memory.")

    # Turn 2: Recall the information in a new session
    print("\n--- Turn 2: Recalling Information ---")
    runner2 = Runner(
            # Use the second agent, which has the memory tool
            agent=memory_recall_agent,
            app_name=APP_NAME,
            session_service=session_service, # Reuse the same service
            memory_service=memory_service   # Reuse the same service
        )
    sessionID = "sessionRetrieval11"
    await runner2.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=sessionID)
    user_input2 = Content(parts=[Part(text="What is my destination I want to travel, what are my flight company and hotel options?")], role="user")

    # Run the second agent
    final_response_text_2 = "(no final response)"
    async for event in runner2.run_async(user_id=USER_ID, session_id=sessionID, new_message=user_input2):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text_2 = event.content.parts[0].text
    print(f"Agent 2 Response: {final_response_text_2}")

import asyncio

# To run this example, you can use the following snippet:
run_scenario()

await run_scenario()

In [ ]:
runner1 = Runner(
        agent=root_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service) # Memory service is now available!
session1 = await run_session_2(runner1,"Hi, my travel plans are: origin:Manchester,  destination:Paris, date_from:1st of Juneh 2026 date_to: 10th of June 2026",
                                 session_id="state_demo_session") # Session ID

In [ ]:
## This is the key method!
##await memory_service.add_session_to_memory("state_demo_session")
#session_test= await session_service.get_session(app_name=runner1.app_name,
#    user_id=USER_ID,
#    session_id="state_demo_session",
#)

#print("✅ Retrieve session from memory")
#session_test.events

In [ ]:
runner2 = Runner(
        # Use the second agent, which has the memory tool
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service, # Reuse the same service
        memory_service=memory_service   # Reuse the same service
    )
await run_session_2(runner2,"Where I want to travel",
                                 "state_demo_session") # Session ID

In [ ]:
#session1 = await session_test.get_session(
#    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
#)
print(session_test.events, session_test.state)

# Let's see what's in the session
print("📝 Session contains:")
for event in session_test.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

# ------------------------------------------
#Testing the state, session, events and debugging the memory
# ------------------------------------------

#### Check a stateless-amnesia agent

In [ ]:
# Turn 1
await run_session_2(
    runner1,
    "Hi, I would like to travel to Japan.",
    "demo",
)

# Turn 2
await run_session_2(
    runner1,
    "Which are the 3 cheapest flight companies and 3 decent hotels",
    "demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session_2(
    runner1,
    "Where do I want to travel?",
    "demo",
)

In [ ]:
# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session_2(
    runner1,
    [
       # "Hi, I am Constantinos and I want to travel to Japan. Could you find some flights less than 2000GBP and hotels 200GBP per night?",
        "Hello! Where I want to travel and what are my hotel options?",  # This time, the agent should remember!
    ],
    "travel-session-singe-agentw",
)

Creating an Agent with Session State Tools and test the agent before applying to multi-agent framework

In [ ]:
response = await runner2.run_debug("Run the trip checker on Flights, Price, and Hotels for Japan trip from Manchester?")

In [ ]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

#### 3.1  Upgrading to a team of specialists to squatt for restaurants and sights around the hotel chosen¶
Follow the strategy as below.

1. Awaiting for a hotel approval from user
2. Have a parallel agent to run the subagents
3. Restaurant looker
4. Sights agent
5. Use the daily planner as coordinator to:
6. Utilise the MCP tool (google maps) and call the above agents

Summarise the findings and store them in the Database

Follow the documentation and generate an API key for [google maps](https://console.cloud.google.com/google/maps-apis/credentials?project=gen-ai-course-476317) to use Model Context Protocol [MCP-tools](https://google.github.io/adk-docs/tools-custom/mcp-tools/#step-2-implement-the-server-logic) calls to find restaraunts close to the hotel of the destination



In [ ]:
# Configure the Gemini API key
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# Retrieve the API key from an environment variable or directly insert it.
# Using an environment variable is generally safer.
# Ensure this environment variable is set in the terminal where you run 'adk web'.
# Example: export GOOGLE_MAPS_API_KEY="YOUR_ACTUAL_KEY"
google_maps_api_key = os.environ.get("GOOGLE_MAPS_API_KEY")

try:
    GOOGLE_MAPS_API_KEY = UserSecretsClient().get_secret("GOOGLE_MAPS_API_KEY")
    os.environ["GOOGLE_MAPS_API_KEY"] = GOOGLE_MAPS_API_KEY
    print("✅ GOOGLE MAPS API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: 'GOOGLE_API_KEY' failed to be added to the Kaggle secrets. Details: {e}")

if not google_maps_api_key:
    # Fallback or direct assignment for testing - NOT RECOMMENDED FOR PRODUCTION
    google_maps_api_key = "YOUR_GOOGLE_MAPS_API_KEY_HERE" # Replace if not using env var
    if google_maps_api_key == "YOUR_GOOGLE_MAPS_API_KEY_HERE":
        print("WARNING: GOOGLE_MAPS_API_KEY is not set. Please set it as an environment variable or in the script.")
        # You might want to raise an error or exit if the key is crucial and not found.

In [ ]:
# define the helper function to manage the complete conversation session,
#handling session creation/retrieval, query processing,
#and response streaming. It supports both single queries
#and multiple queries in sequence.
# Define helper functions that will be reused throughout the notebook
async def run_session_1(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

In [ ]:
maps_assistant_agent = LlmAgent(
    model='gemini-2.0-flash',
    name='maps_assistant_agent',
    instruction='Help the user with mapping, directions, and finding places using Google Maps tools.',
    tools=[
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params = StdioServerParameters(
                    command='npx',
                    args=[
                        "-y",
                        "@modelcontextprotocol/server-google-maps",
                    ],
                    # Pass the API key as an environment variable to the npx process
                    # This is how the MCP server for Google Maps expects the key.
                    env={
                        "GOOGLE_MAPS_API_KEY": google_maps_api_key
                    }
                ),
            ),
            # You can filter for specific Maps tools if needed:
            # tool_filter=['get_directions', 'find_place_by_id']
        )
    ],
)

In [ ]:
# Memory recall agent
memory_recall_agent = Agent(
    model=model_config,
    name="MemoryRecallAgent",
    instruction="Use the 'load_memory' tool "
                "the answer might be in past conversations.",
    tools=[load_memory],
    sub_agents= [maps_assistant_agent],
)

In [ ]:
runner1 = Runner(
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service) # Memory service is now available!
session1 = await run_session_2(runner1,"Find me restaurant less than 0.5 mile close to cheapest hotel out of the 3 saved in the my_agent_data.db",
                                 session_id="sessionRetrieval11") # Session ID

In [ ]:
    runner2 = Runner(
            # Use the second agent, which has the memory tool
            agent=memory_recall_agent,
            app_name=APP_NAME,
            session_service=session_service, # Reuse the same service
            memory_service=memory_service   # Reuse the same service
        )
    sessionID = "sessionRetrieval11"
    await runner2.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=sessionID)
    user_input2 = Content(parts=[Part(text="What is my destination I want to travel, what are my flight company and hotel options?")], role="user")


In [ ]:
# Define a runner
runner_map = InMemoryRunner(agent=maps_assistant_agent)
print("✅ Runner created.")

response = await runner_map.run_debug("Find me restaurant less than 0.5 mile close to Augustus Hotel Riccione", verbose=False)

In [ ]:
runner_map = Runner(agent=maps_assistant_agent,app_name="maps", session_service=session_service)
await run_session_1(runner_map,
             ["Could you suggest me good a pizza restaurant less than 0.25 mile to Napoleon Hotel"],
                   "test")

In [ ]:
#Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

## Section 4. Summary
✅ Enjoy your trip!
* Built a multi-agent for travel  with ADK!

### 5. Future work
* Context compaction since the the session database will quickly add up.
* Using a Booking Agent
* Evaluation


#### Resources

1. Addison Howard, Brenda Flynn, Eric Schmidt, Kanchana Patlolla, Kinjal Parekh, María Cruz, Naz Bayrak, Polong Lin, and Ray Harvey. Agents Intensive - Capstone Project. https://kaggle.com/competitions/agents-intensive-capstone-project, 2025. Kaggle.
2. [ADK-Docs](https://google.github.io/adk-docs/)
3. [ADK-MCP tools](https://google.github.io/adk-docs/tools-custom/mcp-tools/#2-building-an-mcp-server-with-adk-tools-mcp-server-exposing-adk)
4. [ADK_Python](https://github.com/google/adk-python)
5. [ADK-setting-up-environments](https://google.github.io/adk-docs/grounding/google_search_grounding/#set-up-environment-install-adk)

https://www.kaggle.com/code/jocelyndumlao/ai-powered-task-manager-with-automation
https://www.kaggle.com/code/sumakasula/multi-agent-trip-planner
https://www.kaggle.com/code/vimmyverma/travel-tactics
https://www.kaggle.com/code/gajuatulasi/smart-trip-planner
https://www.kaggle.com/code/vibharai28/travel-planning-agent-for-japan-trip
https://www.kaggle.com/code/ghowe2ai/automated-educational-agent-platform

https://google.github.io/adk-docs/sessions/memory/#example-using-two-memory-services https://www.kaggle.com/learn-guide/5-day-agent